# Understand paper from scratch

In [34]:
with tf.Session() as sess:
  devices = sess.list_devices()
devices

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 17681910358801304916),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10858605268217959304),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 701322241902570240),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 7927899751, 406705154930552638)]

In [1]:
## notebook setup
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import pickle
import datetime

In [3]:
import sys

sys.path.append('./libs/')
from parameters import *

sys.path.append(DATA_PATH)
sys.path.append('./CDN_Molecule/')

import preProcess
import model

#from challenge_setup import *

# Preprocessing

In [4]:
FLAGS = tf.app.flags.FLAGS

## Loading data
 
The dataset are 250k randomly chosen drug-like molecules from the ZINC database.
 

In [5]:
# Load data, by default from data/TrainVectors.pickle
print(f"Loading data from file '{FLAGS.data_file}' ...")
x = np.array(pickle.load(open(FLAGS.data_file, "rb")))
print("done")

Loading data from file '/home/fabsta/projects/deeplearning/vae_playground/CDN_Molecule/data/TrainVectors.pickle' ...
done


In [6]:
x.shape
print(f" We have {x.shape[0]} molecules, with {x.shape[1]} features each")

 We have 173196 molecules, with 50 features each


## Interpretation of input

In [7]:
x[:2]


array([[34,  0,  1,  0,  2,  2,  3,  4,  5,  0,  0,  6,  7,  8,  9,  8,
        10,  8,  8,  6, 11,  8, 12, 10, 10,  8, 10, 12,  0, 13,  8,  9,
        13,  0,  1,  0,  2,  2,  3,  4,  6,  0, 13,  0,  5, 35, 36, 36,
        36, 36],
       [34,  0,  0,  1,  7,  3, 19,  4,  6,  0,  0, 13,  1,  0,  2,  4,
         6,  0, 13,  6,  0,  0, 13,  1,  0,  2,  3,  4,  6, 15, 13,  8,
         5,  8, 20,  8,  8,  5, 21, 22, 35, 36, 36, 36, 36, 36, 36, 36,
        36, 36]], dtype=int32)

## Shuffle data

In [8]:
# Shuffle data -- ok, shuffle indices of rows and create a copy, it's memory effective
np.random.seed(10)
print(f"np.arange(len(x)) = {np.arange(len(x))}\nlen(x) = {len(x)}")
shuffle_indices = np.random.permutation(np.arange(len(x)))
x_shuffled = x[shuffle_indices]

np.arange(len(x)) = [     0      1      2 ... 173193 173194 173195]
len(x) = 173196


## Train/Test split

In [9]:
print(f"-1 * int({FLAGS.dev_sample_percentage}) * float({x_shuffled.shape[0]})")
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(x_shuffled.shape[0]))
print(f"x_shuffled[:{dev_sample_index}], x_shuffled[{dev_sample_index}:]")
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
print(("Train/Dev split: {:d}/{:d}".format(len(x_train), len(x_dev))))

-1 * int(0.03) * float(173196)
x_shuffled[:-5195], x_shuffled[-5195:]
Train/Dev split: 168001/5195


In [10]:
def split_input(batch):
    """
    This looks like "shifting" each row (instance) by one position to the left and adding 36 at the end,
    for example
    [34  0 16  0  6  0 13  0 10  5  8  6 16 15 13  8  9  8  6 10  8 12 10  9
      0  0  0  7 12  8  9  8  8  8  8  8  9 15  0 13 10  6  0 13  8  5 16 15
     35 36]
 
    becomes
 
    [ 0 16  0  6  0 13  0 10  5  8  6 16 15 13  8  9  8  6 10  8 12 10  9  0
      0  0  7 12  8  9  8  8  8  8  8  9 15  0 13 10  6  0 13  8  5 16 15 35
     36 36]
 
    :param current_batch_as_ndarray: 64 rows -- 64 instances per batch, each instance has 50 dimensions
    :return: tuple: the parameter itself (current_batch_as_ndarray) and the shifted one, see above
    """
 
    # There are 64 rows -- 64 instances per batch, each instance has 50 dimensions
    # print("split_input(), current_batch_as_ndarray.shape:", current_batch_as_ndarray.shape)
 
    # simply copy current_batch_as_ndarray to x_batch output
    x_batch = batch
 
    # remove the first position and add 36 to the end
    y_batch = np.concatenate([x_batch[:, 1:], np.zeros(shape=[x_batch.shape[0], 1], dtype=np.int32) + 36],
                             axis=1)
 
    return x_batch, y_batch

In [11]:
x_batch, y_batch = split_input(x_train[:1])


# The model

In [12]:
import tensorflow as tf
 
 
class CDN:
 
    def __init__(self, sequence_length, vocab_size, embedding_size, filter_sizes, num_filters, max_molecule_length,
                 gaussian_samples, variational=True, l2_reg_lambda=0.5, generation_mode=False, test_mode=False):
# define some variables
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.max_molecule_length = max_molecule_length
        self.l2_reg_lambda = l2_reg_lambda
        self.gaussian_samples_dim = gaussian_samples
        self.variational = variational
        self.encoder_input_GO = tf.placeholder(tf.int32, [None, sequence_length], name="encoder_input")
        self.encoder_input = tf.placeholder(tf.int32, [None, sequence_length], name="encoder_input")  ## no go
        self.gaussian_samples = tf.placeholder(tf.float32, [None, self.gaussian_samples_dim], name="unit_gaussians")
        self.generation_mode = generation_mode
        self.test_mode = test_mode
# how to encode
    def encode(self):
        # Embedding layer
        with tf.name_scope("embedding"):
            self.E = tf.Variable(tf.random_uniform([self.vocab_size, self.embedding_size], -1.0, 1.0), name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.E, self.encoder_input)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
            self.embedded_chars_go = tf.nn.embedding_lookup(self.E, self.encoder_input_GO)
 
        # Create a convolution layers for each filter size
        conv_flatten = []
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[self.num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                conv_flatten.append(tf.contrib.layers.flatten(h))
        conv_output = tf.concat(conv_flatten, axis=1)
 
        # Flatten feature vector
        h_pool_flat3 = tf.nn.relu(tf.contrib.layers.linear(conv_output, 450))
 
        if self.variational:
            with tf.name_scope("Variational"):
                self.z_mean = tf.contrib.layers.linear(h_pool_flat3, 300)
                self.z_stddev = tf.contrib.layers.linear(h_pool_flat3, 300)
                latent_loss = 0.5 * tf.reduce_sum(
                    tf.square(self.z_mean) + tf.square(self.z_stddev) -
                    tf.log(tf.square(self.z_stddev) + 1e-10 ) - 1,
                    1
                )
                self.mean_latent_loss = tf.reduce_mean(latent_loss)
                if self.generation_mode:
                    h_pool_flat = self.gaussian_samples
                else:
                    h_pool_flat = self.z_mean + (self.z_stddev * self.gaussian_samples)
 
                h_pool_flat = tf.identity(h_pool_flat, "encoded_final")
 
        return h_pool_flat, self.mean_latent_loss
 
    def decode_rnn(self, z):
        def pick_next_argmax(former_output, step):
            next_symbol = tf.expand_dims(tf.stop_gradient(tf.argmax(former_output, 1)), axis=-1)
            return tf.nn.embedding_lookup(self.E, next_symbol), next_symbol
 
        def pick_next_top_k(former_output, step):
            next_symbol = tf.multinomial(former_output, 1)
            return tf.nn.embedding_lookup(self.E, next_symbol), next_symbol
 
        with tf.name_scope("Decoder"):
            self.decode_start = tf.nn.relu(tf.contrib.layers.linear(z, 150))
            decoder_inputs_list = tf.split(self.embedded_chars_go, self.max_molecule_length, axis=1)
            decoder_inputs_list = [tf.squeeze(i, axis=1) for i in decoder_inputs_list]
            rnn_cell = tf.nn.rnn_cell.LSTMCell(150, state_is_tuple=False)
 
            self.lstm_outputs = []
            temp_logits = []
            self.all_symbols = []
            symbol = tf.ones(1)  # output for test mode
            for i in range(self.max_molecule_length):
                if not self.test_mode or i == 0:
                    if i == 0:
                        output, state = rnn_cell(decoder_inputs_list[i], state=z)
                    else:
                        output, state = rnn_cell(decoder_inputs_list[i], state=state)
                else:
                    next_decoder_input, symbol = pick_next_argmax(temp_logits[-1], i)
                    next_decoder_input = tf.squeeze(next_decoder_input, axis=1)
                    output, state = rnn_cell(next_decoder_input, state=state)
                with tf.variable_scope("decoder_output_to_logits") as scope_logits:
                    if i > 0:
                        scope_logits.reuse_variables()
                    temp_logits.append(tf.contrib.layers.linear(output, self.vocab_size))
 
                self.lstm_outputs.append(output)
                if i > 0:
                    self.all_symbols.append(symbol)
                if i == self.max_molecule_length - 1 and self.test_mode:
                    self.all_symbols.append(pick_next_argmax(temp_logits[-1], i)[1])
            if self.test_mode:
                self.all_symbols = tf.squeeze(tf.transpose(tf.stack(self.all_symbols), [1,0,2]), axis=-1)
 
            self.decoder_logits = tf.transpose(tf.stack(temp_logits), perm=[1, 0, 2])
            self.decoder_prediction = tf.argmax(self.decoder_logits, 2, name="decoder_predictions")
 
            return self.decoder_logits
 
    def loss(self, logits, latent_loss):
        with tf.name_scope("loss"):
            self.output_onehot = tf.one_hot(self.encoder_input, self.vocab_size)
            self.losses = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.output_onehot)
            self.CE_loss = tf.reduce_mean(self.losses)
            self.total_loss = self.CE_loss + .00001 * latent_loss
 
        with tf.name_scope("accuracy"):
            decoder_prediction = tf.argmax(logits, 2, name="decoder_predictions")
            x_target = tf.to_int64(self.encoder_input)
            correct_predictions = tf.equal(decoder_prediction, x_target)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
 
        return self.total_loss, self.accuracy


In [13]:
cnn = model.CDN(
            sequence_length=x_train.shape[1],
            vocab_size=FLAGS.vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            max_molecule_length=FLAGS.max_molecule_length,
            gaussian_samples=FLAGS.unit_gaussian_dim,
            l2_reg_lambda=FLAGS.l2_reg_lambda,
            variational=True,
            test_mode=False,
            generation_mode=False
        )

# Training
 
What the code does
 
1. instantiate model
2. set learning rate
3. set adam optimizer
4. Instantiate Encoder
5. Instantiate Decoder
6. Instantiate Loss function (optimize gradients)
7. Define summaries
8. Generate batches
9. Run training over all batches

In [36]:
# in the scope of a default TF graph... why? TL;DR: It's unnecessary, but it's a good practice to follow.
# See https://stackoverflow.com/a/39616491
with tf.Graph().as_default():
    sess_config = tf.ConfigProto(
        gpu_options=tf.GPUOptions(allow_growth=True)
    )
    sess = tf.Session(config=sess_config)
    # here we begin the tensorflow session
    with sess.as_default():
        cnn = model.CDN(
            sequence_length=x_train.shape[1],
            vocab_size=FLAGS.vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            max_molecule_length=FLAGS.max_molecule_length,
            gaussian_samples=FLAGS.unit_gaussian_dim,
            l2_reg_lambda=FLAGS.l2_reg_lambda,
            variational=True,
            test_mode=False,
            generation_mode=False
        )
        # cnn is a simple CDN object, nothing inherited from TF classes
        assert isinstance(cnn, model.CDN)
 
        # TODO why do we need a global_step non-trainable variable?
        # According to this StackOverflow, it's a way to train AdamOptimizer with custom learning rate, see
        # https://stats.stackexchange.com/questions/200063/adam-optimizer-with-exponential-decay
        global_step = tf.Variable(1, name="global_step", trainable=False)
 
        # A scalar `Tensor` of the same type as `learning_rate`.  The decayed learning rate.
        learning_rate = tf.train.exponential_decay(learning_rate=FLAGS.initial_learning_rate, global_step=global_step,
                                                   decay_steps=5000, decay_rate=0.96, staircase=True)
 
        # somehow confusing how the learning rate is passed to the AdamOptimizer...
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # but we know the type: it's AdamOptimizer instance, inherited from Optimizer
        assert isinstance(optimizer, tf.train.AdamOptimizer)
 
        # CDN.encode() defines the computational graph - returns two tensors
        # TODO explore encode() in detail
       
        #####
        # Encoding part
        #####
        # returns pooling and loss function (mean_latent_loss)
        encoded, latent_loss = cnn.encode()
        assert isinstance(encoded, tf.Tensor)
        assert isinstance(latent_loss, tf.Tensor)
       
        #####
        # Decoder part
        #####
        # another Tensor output from the decoder
        logits = cnn.decode_rnn(encoded)
        assert isinstance(logits, tf.Tensor)
 
        #####
        # loss function
        #####
        # and CDN.loss() returns two Tensors: loss and accuracy
        loss, accuracy = cnn.loss(logits, latent_loss)
        assert isinstance(loss, tf.Tensor)
        assert isinstance(accuracy, tf.Tensor)
 
        # grads_and_vars is a list
        # It returns a list of (gradient, variable) pairs where "gradient" is the gradient for "variable".
        # It is the first part of minimize()
        grads_and_vars = optimizer.compute_gradients(loss)
 
        # The second part of minimize() which applies gradients to variables
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        assert isinstance(train_op, tf.Operation)
 
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)
 
        # Output directory
        timestamp = str(int(time.time()))
        if FLAGS.parameters_file is not None:
            out_dir = os.path.abspath(os.path.join(os.path.curdir, FLAGS.parameters_file))
        else:
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print(("Writing to {}\n".format(out_dir)))
 
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", loss)
        acc_summary = tf.summary.scalar("accuracy", accuracy)
        kl_div_summary = tf.summary.scalar("KL-div", latent_loss)
        CE_loss_summary = tf.summary.scalar("CE-loss", cnn.CE_loss)
        learning_rate_summary = tf.summary.scalar("learningRate", learning_rate)
 
        # Train Summaries
        train_summary_op = tf.summary.merge(
            [loss_summary, acc_summary, grad_summaries_merged, kl_div_summary, CE_loss_summary,
             learning_rate_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train") # file directory
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph) # save in file
 
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary, kl_div_summary, CE_loss_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
 
        # Checkpoint directory.
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)
       
        # Resuming training
        if (FLAGS.load_param) and (FLAGS.parameters_file is not None):
            # model file to restart training
            checkpoint_file = FLAGS.parameters_file
            saver.restore(sess, checkpoint_file)
            print("Loaded: " + str(checkpoint_file))
        else:
            sess.run(tf.global_variables_initializer())
 
        # Generate batches - a batch iterator for a data set
        num_epochs  = 1
        # Simulate 1 batch and 1 epoch
        #batches = preProcess.batch_iter(x_train, 168001, 1)
        batches = preProcess.batch_iter(x_train, FLAGS.batch_size, FLAGS.num_epochs)
        # what is actually 'batches'? - it is a generator
        # a batch is defined as an x and a y.
       
        for idx, batch in enumerate(batches):
            current_step = tf.train.global_step(sess, global_step)
            # split each batch into x and y
            #print(f"splitting batch {batch.size} ")

            x_bat, y_bat = split_input(batch)
            # feed data to placeholders, so data that changes with each batch
            #print(f"starting batch {idx} ")
            feed_dict = {
                # these are CDN placeholders
                cnn.encoder_input: y_bat,
                cnn.encoder_input_GO: x_bat,
                cnn.gaussian_samples: np.random.normal(size=[x_bat.shape[0], FLAGS.unit_gaussian_dim]),
            }
 
            # Runs operations and evaluates tensors in `fetches`.
            # TODO Why the heck do we need to call train_op AND total_loss, summary, etc.? Just for debugging?
            _, step, summaries, loss, accuracy, kldiv, CEloss = sess.run(
                fetches=[train_op, global_step, train_summary_op, cnn.total_loss, cnn.accuracy, cnn.mean_latent_loss,
                         cnn.CE_loss], feed_dict=feed_dict)
 
            time_str = datetime.datetime.now().isoformat()
            if current_step % 100 == 0:
                print(("{}: step {}, loss {:g}, latentLoss: {:g}, reconstructionLoss: {:g}, acc {:g}".format(
                    time_str, step, loss, kldiv, CEloss, accuracy)))
 
            train_summary_writer.add_summary(summaries, step)
 
            # every N steps we run also evaluation
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
 
                x_bat, y_bat = split_input(batch)
                feed_dict = {
                    cnn.encoder_input: y_bat,
                    cnn.encoder_input_GO: x_bat,
                    cnn.gaussian_samples: np.random.normal(size=[x_bat.shape[0], FLAGS.unit_gaussian_dim]),
                }
 
                step, summaries, loss, accuracy, kldiv, CEloss = sess.run(
                    [global_step, dev_summary_op, cnn.total_loss, cnn.accuracy, cnn.mean_latent_loss, cnn.CE_loss],
                    feed_dict)
 
                time_str = datetime.datetime.now().isoformat()
                print(
                    ("{}: step {}, loss {:g}, klDiv: {:g}, CE-loss: {:g}, acc {:g}".format(time_str, step, loss, kldiv,
                                                                                           CEloss, accuracy)))
                if dev_summary_writer:
                    dev_summary_writer.add_summary(summaries, step)
 
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print(("Saved model checkpoint to {}\n".format(path)))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2018-11-21T16:20:16.364127: step 1501, loss 0.32642, latentLoss: 771.36, reconstructionLoss: 0.318706, acc 0.892187

Evaluation:
2018-11-21T16:20:16.378448: step 1501, loss 0.322542, klDiv: 766.123, CE-loss: 0.314881, acc 0.894375

2018-11-21T16:20:33.086793: step 1601, loss 0.306876, latentLoss: 759.483, reconstructionLoss: 0.299282, acc 0.900937

Evaluation:
2018-11-21T16:20:33.103466: step 1601, loss 0.29961, klDiv: 764.292, CE-loss: 0.291967, acc 0.904688

2018-11-21T16:20:49.696861: step 1701, loss 0.309802, latentLoss: 769.577, reconstructionLoss: 0.302107, acc 0.895

Evaluation:
2018-11-21T16:20:49.710649: step 1701, loss 0.289741, klDiv: 777.01, CE-loss: 0.281971, acc 0.90625

2018-11-21T16:21:06.280753: step 1801, loss 0.29437, latentLoss: 741.794, reconstructionLoss: 0.286952, acc 0.898125

Evaluation:
2018-11-21T16:21:06.297168: step 1801, loss 0.274604, klDiv: 736.772, CE-loss: 0.267237, acc 0.905937

2018-11-21T16:21:22.823052: step 1901, loss 0.267026, latentLoss: 735.651

Saved model checkpoint to /home/fabsta/projects/deeplearning/vae_playground/runs/1542813331/checkpoints/model-5000

2018-11-21T16:30:00.683778: step 5101, loss 0.125073, latentLoss: 681.057, reconstructionLoss: 0.118262, acc 0.959375

Evaluation:
2018-11-21T16:30:00.695056: step 5101, loss 0.11944, klDiv: 673.486, CE-loss: 0.112705, acc 0.961563

2018-11-21T16:30:16.611417: step 5201, loss 0.11876, latentLoss: 674.292, reconstructionLoss: 0.112017, acc 0.962812

Evaluation:
2018-11-21T16:30:16.624710: step 5201, loss 0.109415, klDiv: 678.153, CE-loss: 0.102633, acc 0.9675

2018-11-21T16:30:32.566839: step 5301, loss 0.124348, latentLoss: 676.501, reconstructionLoss: 0.117583, acc 0.961875

Evaluation:
2018-11-21T16:30:32.580599: step 5301, loss 0.117604, klDiv: 673.193, CE-loss: 0.110872, acc 0.964375

2018-11-21T16:30:48.481325: step 5401, loss 0.123392, latentLoss: 658.309, reconstructionLoss: 0.116808, acc 0.960625

Evaluation:
2018-11-21T16:30:48.490213: step 5401, loss 0.116591, k

2018-11-21T16:39:16.919304: step 8601, loss 0.0777792, latentLoss: 595.371, reconstructionLoss: 0.0718254, acc 0.975312

Evaluation:
2018-11-21T16:39:16.928865: step 8601, loss 0.0660094, klDiv: 594.918, CE-loss: 0.0600602, acc 0.979688

2018-11-21T16:39:32.739754: step 8701, loss 0.0836766, latentLoss: 582.154, reconstructionLoss: 0.0778551, acc 0.973437

Evaluation:
2018-11-21T16:39:32.749445: step 8701, loss 0.0766376, klDiv: 581.863, CE-loss: 0.0708189, acc 0.977813

2018-11-21T16:39:48.561852: step 8801, loss 0.0623446, latentLoss: 566.272, reconstructionLoss: 0.0566819, acc 0.980937

Evaluation:
2018-11-21T16:39:48.575333: step 8801, loss 0.0555499, klDiv: 564, CE-loss: 0.0499099, acc 0.984062

2018-11-21T16:40:04.343917: step 8901, loss 0.0908148, latentLoss: 588.216, reconstructionLoss: 0.0849326, acc 0.966563

Evaluation:
2018-11-21T16:40:04.360443: step 8901, loss 0.0798287, klDiv: 586.823, CE-loss: 0.0739605, acc 0.972812

2018-11-21T16:40:20.154925: step 9001, loss 0.075954

2018-11-21T16:48:15.186110: step 12001, loss 0.0606165, latentLoss: 526.173, reconstructionLoss: 0.0553548, acc 0.983125

Evaluation:
2018-11-21T16:48:15.204445: step 12001, loss 0.0553451, klDiv: 522.86, CE-loss: 0.0501165, acc 0.983125

2018-11-21T16:48:30.941155: step 12101, loss 0.0606653, latentLoss: 566.529, reconstructionLoss: 0.055, acc 0.98

Evaluation:
2018-11-21T16:48:30.952190: step 12101, loss 0.0550465, klDiv: 565.661, CE-loss: 0.0493899, acc 0.983125

2018-11-21T16:48:46.766987: step 12201, loss 0.0587699, latentLoss: 547.127, reconstructionLoss: 0.0532986, acc 0.980937

Evaluation:
2018-11-21T16:48:46.778066: step 12201, loss 0.0486316, klDiv: 548.87, CE-loss: 0.0431429, acc 0.98625

2018-11-21T16:49:02.525807: step 12301, loss 0.0708015, latentLoss: 542.182, reconstructionLoss: 0.0653797, acc 0.975625

Evaluation:
2018-11-21T16:49:02.544475: step 12301, loss 0.056677, klDiv: 543.615, CE-loss: 0.0512408, acc 0.983125

2018-11-21T16:49:18.360224: step 12401, loss 0.05836

...

2018-11-21T18:00:25.708904: step 39501, loss 0.0208382, latentLoss: 404.25, reconstructionLoss: 0.0167957, acc 0.994375

Evaluation:
2018-11-21T18:00:25.721959: step 39501, loss 0.0184161, klDiv: 403.206, CE-loss: 0.0143841, acc 0.994687

2018-11-21T18:00:42.092444: step 39601, loss 0.0173479, latentLoss: 379.516, reconstructionLoss: 0.0135528, acc 0.995938

Evaluation:
2018-11-21T18:00:42.109053: step 39601, loss 0.0161399, klDiv: 378.812, CE-loss: 0.0123518, acc 0.997187

2018-11-21T18:00:58.640328: step 39701, loss 0.0257881, latentLoss: 401.948, reconstructionLoss: 0.0217687, acc 0.992813

Evaluation:
2018-11-21T18:00:58.653271: step 39701, loss 0.0193794, klDiv: 399.469, CE-loss: 0.0153847, acc 0.995625

2018-11-21T18:01:15.037816: step 39801, loss 0.017923, latentLoss: 371.187, reconstructionLoss: 0.0142111, acc 0.996562

Evaluation:
2018-11-21T18:01:15.051193: step 39801, loss 0.0160739, klDiv: 372.881, CE-loss: 0.0123451, acc 0.996562

2018-11-21T18:01:31.459500: step 39901, lo

2018-11-21T18:09:26.332006: step 42901, loss 0.0208502, latentLoss: 367.384, reconstructionLoss: 0.0171763, acc 0.995

Evaluation:
2018-11-21T18:09:26.351111: step 42901, loss 0.0145654, klDiv: 368.223, CE-loss: 0.0108832, acc 0.99625

2018-11-21T18:09:42.040175: step 43001, loss 0.0167949, latentLoss: 398.175, reconstructionLoss: 0.0128132, acc 0.995938

Evaluation:
2018-11-21T18:09:42.055011: step 43001, loss 0.0150072, klDiv: 399.66, CE-loss: 0.0110106, acc 0.996562

2018-11-21T18:09:57.737500: step 43101, loss 0.0287204, latentLoss: 387.427, reconstructionLoss: 0.0248461, acc 0.991875

Evaluation:
2018-11-21T18:09:57.746493: step 43101, loss 0.0236865, klDiv: 388.956, CE-loss: 0.019797, acc 0.992813

2018-11-21T18:10:13.467679: step 43201, loss 0.0173765, latentLoss: 382.559, reconstructionLoss: 0.0135509, acc 0.996875

Evaluation:
2018-11-21T18:10:13.481994: step 43201, loss 0.0143355, klDiv: 379.208, CE-loss: 0.0105435, acc 0.996562

2018-11-21T18:10:29.153264: step 43301, loss 0

2018-11-21T18:18:21.041673: step 46301, loss 0.0193754, latentLoss: 415.464, reconstructionLoss: 0.0152207, acc 0.995

Evaluation:
2018-11-21T18:18:21.059740: step 46301, loss 0.0164862, klDiv: 417.151, CE-loss: 0.0123147, acc 0.995625

2018-11-21T18:18:37.195575: step 46401, loss 0.0190506, latentLoss: 367.508, reconstructionLoss: 0.0153755, acc 0.995938

Evaluation:
2018-11-21T18:18:37.207070: step 46401, loss 0.0148305, klDiv: 367.674, CE-loss: 0.0111538, acc 0.996875

2018-11-21T18:18:53.229189: step 46501, loss 0.0136359, latentLoss: 359.878, reconstructionLoss: 0.0100371, acc 0.996562

Evaluation:
2018-11-21T18:18:53.241293: step 46501, loss 0.010506, klDiv: 358.847, CE-loss: 0.00691757, acc 0.998438

2018-11-21T18:19:09.178288: step 46601, loss 0.0156603, latentLoss: 378.71, reconstructionLoss: 0.0118732, acc 0.995938

Evaluation:
2018-11-21T18:19:09.195236: step 46601, loss 0.0113575, klDiv: 377.358, CE-loss: 0.0075839, acc 0.99875

2018-11-21T18:19:25.521900: step 46701, loss 

2018-11-21T18:27:40.192933: step 49801, loss 0.0208243, latentLoss: 393.74, reconstructionLoss: 0.0168869, acc 0.994687

Evaluation:
2018-11-21T18:27:40.209361: step 49801, loss 0.0164973, klDiv: 393.358, CE-loss: 0.0125637, acc 0.995938

2018-11-21T18:27:55.959700: step 49901, loss 0.013541, latentLoss: 352.611, reconstructionLoss: 0.0100149, acc 0.995625

Evaluation:
2018-11-21T18:27:55.977212: step 49901, loss 0.0129158, klDiv: 350.171, CE-loss: 0.00941409, acc 0.99625

2018-11-21T18:28:11.747051: step 50001, loss 0.0168309, latentLoss: 369.208, reconstructionLoss: 0.0131388, acc 0.994375

Evaluation:
2018-11-21T18:28:11.764156: step 50001, loss 0.0146152, klDiv: 370.846, CE-loss: 0.0109068, acc 0.996875



ResourceExhaustedError: /home/fabsta/projects/deeplearning/vae_playground/runs/1542813331/checkpoints/model-50000.data-00000-of-00001.tempstate10334406296181928179; No space left on device
	 [[node save/SaveV2 (defined at <ipython-input-36-900d8d3add4a>:116)  = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, beta1_power/_337, beta2_power/_339, conv-maxpool-3/W/_341, conv-maxpool-3/W/Adam/_343, conv-maxpool-3/W/Adam_1/_345, conv-maxpool-3/b/_347, conv-maxpool-3/b/Adam/_349, conv-maxpool-3/b/Adam_1/_351, conv-maxpool-4/W/_353, conv-maxpool-4/W/Adam/_355, conv-maxpool-4/W/Adam_1/_357, conv-maxpool-4/b/_359, conv-maxpool-4/b/Adam/_361, conv-maxpool-4/b/Adam_1/_363, conv-maxpool-5/W/_365, conv-maxpool-5/W/Adam/_367, conv-maxpool-5/W/Adam_1/_369, conv-maxpool-5/b/_371, conv-maxpool-5/b/Ada

In [19]:
len(np.array(x_train))

168001

## Evaluation

In [25]:
# Load data
print("Loading data...")
fda_file = "/home/fabsta/projects/deeplearning/vae_playground/CDN_Molecule/data/FDA"
fda_x = np.array(pickle.load(open(fda_file, "rb")))
fda_x.shape

Loading data...


(576, 50)


## Shuffle and split

In [26]:
# Shuffle data
np.random.seed(10)
print("Num of input molecules: ", str(len(fda_x)))
shuffle_indices = np.random.permutation(np.arange(len(fda_x)))
fda_x_shuffled = fda_x[shuffle_indices]

# Split train/test set
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(fda_x)))
fda_x_train, fda_x_dev = fda_x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
print(("Train/Dev split: {:d}/{:d}".format(len(fda_x_train), len(fda_x_dev))))

Num of input molecules:  576
Train/Dev split: 559/17


### Helper functions

In [28]:
def split_input(batch):
    x_batch = batch
    y_batch = np.concatenate([x_batch[:, 1:], np.zeros(shape=[x_batch.shape[0], 1], dtype=np.int32) + 36], axis=1)
    return x_batch, y_batch


def get_smile(ar, num2char):
    smile = ""
    for i in ar:
        smile += num2char[str(i)]
    return smile

def analyze_output(predictions):
    # analyze output
    num2char = preProcess.load_json_file('data/num2char.json')
    num2char['34'] = 'GO' ; num2char['35'] = 'EN' ; num2char['36'] = 'PA'
    count = 0
    new_mol = []#set()
    real_mol = []
    for index, pred in enumerate(predictions):
        real_smile = get_smile(x_bat[index], num2char)
        real_smile = real_smile.split('EN')[0].split('GO')[1]
        fake_smile = get_smile(pred, num2char)
        fake_smile = fake_smile.split('EN')[0]
        res = preProcess.mol_analysis(fake_smile, real_smile)
        count += res
        if res == 1:
            if fake_smile != real_smile:
                # new_mol.add(fake_smile)
                new_mol.append(fake_smile)
                real_mol.append(real_smile)

    return new_mol, real_mol, count

In [32]:
checkpoint_file = '/home/fabsta/projects/deeplearning/vae_playground/runs/1542811459'
# Evaluation
# ==================================================
graph = tf.Graph()
with tf.Graph().as_default():
    sess_config = tf.ConfigProto(
        gpu_options=tf.GPUOptions(allow_growth=True)
    )
    sess = tf.Session(config=sess_config)
    with sess.as_default():
        CDN = model.CDN(
            sequence_length=x_train.shape[1],
            vocab_size=FLAGS.vocab_size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            max_molecule_length=FLAGS.max_molecule_length,
            gaussian_samples=FLAGS.unit_gaussian_dim,
            l2_reg_lambda=FLAGS.l2_reg_lambda,
            variational=True,
            test_mode=True,
        )

        encoded, latent_loss = CDN.encode()
        logits = CDN.decode_rnn(encoded)
        loss, accuracy = CDN.loss(logits, latent_loss)

        #checkpoint_file = FLAGS.parameters_file
        saver = tf.train.Saver().restore(sess, checkpoint_file)
        print("restored " + str(checkpoint_file))

        x_bat, y_bat = split_input(x_dev)
        feed_dict = {
            CDN.encoder_input: y_bat,
            CDN.encoder_input_GO: x_bat,
            CDN.gaussian_samples: np.random.normal(size=[x_bat.shape[0], FLAGS.unit_gaussian_dim], scale=1.0),
        }

        outputs = sess.run(
            [encoded, latent_loss, logits, loss, accuracy, CDN.all_symbols], feed_dict=feed_dict)

        predictions = np.argmax(outputs[2], axis=2)
        # predictions = outputs[-1]

        new_mol, real_mol, valid = analyze_output(predictions)
        with open(FLAGS.output_file, 'w') as f:
            for ori, gen in zip(real_mol, new_mol):
                f.write(ori + ',' + gen + '\n')

        print("Loss " + str(outputs[3]))
        print("Acc with respect to real: " + str(outputs[4]))
        print("Total evaluated molecules: " + str(len(predictions)))
        print("Total valid molecules: " + str(valid))
        print("Total valid molecules ratio: " + str(valid/float(len(predictions))))
        print("New mols: " + str(len(set(new_mol))))

INFO:tensorflow:Restoring parameters from /home/fabsta/projects/deeplearning/vae_playground/runs/1542811459


DataLossError: Unable to open table file /home/fabsta/projects/deeplearning/vae_playground/runs/1542811459: Failed precondition: /home/fabsta/projects/deeplearning/vae_playground/runs/1542811459; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
	 [[node save/RestoreV2 (defined at <ipython-input-32-945db3675af7>:29)  = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/fabsta/.conda/envs/cdn_molecule/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/fabsta/.conda/envs/cdn_molecule/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/fabsta/.conda/envs/cdn_molecule/lib/pytho